In [286]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout
import tarfile
import os
import numpy as np

In [12]:
# Unzip Dataset
  
# open file
file = tarfile.open('20news-group-dataset/20news-19997.tar.gz')
  
# extracting file
file.extractall('./')
  
file.close()

In [468]:
# os.chdir('20news-group-dataset/20_newsgroups/talk.politics.misc')
files = []
for i in range(300):
    with open(os.listdir()[i], 'r') as f:
           files.append(f.read())
processed_text = []  
for text_doc in files:
    text = text_doc.split('\n')
    start = False
    for line in text:
        if 'writes:' in line:
            start = True
            continue
        if start == True:
            words = line.split()
            without_nonalpha = ["".join([char.lower() for char in word if char.isalpha() or char.isnumeric()]) for word in words]
            without_empties = list(filter(lambda x: x != '' , without_nonalpha))
            if without_empties:
                processed_text.append(without_empties)
            


In [406]:
tokenizer = Tokenizer()


def get_tokenized_sequences(corpus):
    tokenizer.fit_on_texts(corpus)
    num_words = len(tokenizer.word_index) +1
    
    input_seq = []
    for line in corpus:
        sequences = tokenizer.texts_to_sequences(line)
        for i in range(1, len(sequences)+1):
            input_seq.append(sequences[:i])
            
            
    return input_seq, num_words
    

In [407]:
def fit_labels(sequences):
    label = []
    for i in range(1, len(sequences)):
        if len(sequences[i]) >= len(sequences[i-1]):
            label.append(sequences[i][len(sequences[i])-1])
        
    return label
        

In [469]:
sequences, num_words = get_tokenized_sequences(processed_text)

padded_sequences = pad_sequences(sequences, padding='pre')

max_len = len(padded_sequences[0])-1

targets = padded_sequences[:,-1]
features = padded_sequences[:, :-1]

targets = targets.reshape(-1,)
targets = tf.keras.utils.to_categorical(targets, num_classes=num_words)
features = features.reshape(len(padded_sequences), max_len,)



In [470]:
model = Sequential()

model.add(Embedding(num_words, 10, input_length = max_len))
model.add(LSTM(50))
model.add(Dropout(0.1))
model.add(Dense(num_words, activation='softmax'))

In [471]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 37, 10)            83240     
_________________________________________________________________
lstm_15 (LSTM)               (None, 50)                12200     
_________________________________________________________________
dropout_8 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 8324)              424524    
Total params: 519,964
Trainable params: 519,964
Non-trainable params: 0
_________________________________________________________________


In [472]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [473]:
model.fit(features, targets, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [475]:
def generate(words, num_words, model, sequence_len):

    for i in range(num_words):
        sequence = pad_sequences(tokenizer.texts_to_sequences([words]), maxlen = sequence_len, padding='pre')
    
        word = tokenizer.index_word[np.argmax(model.predict(sequence), axis=-1)[-1]]
        
        words += ' ' + word
    
    return words


In [489]:
generate('', 35, model, len(padded_sequences[0]))

'soldier is that the us and the slave states did not been sustained for the same reich vetos given free by the politicians prefer naziplutonium under the same reich early that costs the talk of the'